# 0 Functions

In [2]:
%run "0k Benchmark Imports CMZ.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16
/home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
/home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
1.23.1 Numpy


# Part 8: Dendograms

In [4]:

#CO 3-2 all resses starting at 4.3


Num=5
Overlaps=0#4
Ram_Limiter=1#What percent of the cube my ram can handle
LineN="CMZ_CO_J3_2_all_NEW"
Name = "CMZ_CO_J3_2_all_NEW"
name = "CMZ_CO_J3_2_CM_Contours_all_NEW.jpeg"

Abs_Levels = ["All","None","No Clusters", "None (m)", "None All Channels"]



iterations = 6
iter_factor = 1/5
#Line_Name = '_CMZ_CO_J3_2_4.3_start'

NM= False

Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"

Edge_cases=True

if(Edge_cases):
    Edge_title="Edges"
else:
    Edge_title=''
    
Line_Name = '_CMZ_CO_J3_2_4.3_start'

if(NM):
    Line_Name_Save = Line_Name+'_NM'+Smoothe_title+Edge_title
else:
    Line_Name_Save=Line_Name+Smoothe_title+Edge_title



ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution
Min_beam_req = 1/5

Params = [iterations,iter_factor,ovs,min_vel,FOV,Min_res,Min_beam_req]
savePath='/home/ben/Documents/Grad Stuff/MM data/Result Files'
if(NM):
    
    Name_File = "CMZ_Names_New_All_Kinds_NM"+Smoothe_title+Edge_title
else:
    
    Name_File= "CMZ_Names_New_All_Kinds"+Smoothe_title+Edge_title

np.savetxt(Line_Name+"_Params", Params,fmt='%s')


cen_p1 = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
cen_p2 = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')



In [3]:
savePath='./Result Files'



In [ ]:
#Krieger

#Beam
Abs_Level = Abs_Levels[0]



if gal =="GC":
    pathCont = '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits'#'CMZ_Continuum_Splice.fits'
else:
    pathCont = 'Continuum_Reproject.fits'





start_num=0
km=0
for kl in range(5,6):
      
    Prime_Beam = Min_res*((kl*iter_factor+1))#
    max_size = Prime_Beam*7   
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        km=5
        FOVp=[200,800]
        max_size = 20#pc
        if(Match_to_HCO):
            Prime_Beam = 4.3*u.pc#Nico
            FOVp=[70,360]

    else:
        FOVp=FOV
    print(Prime_Beam)
    
    
    vel_prime = min_vel*((km*iter_factor+1))

    if(km==5):
        vel_prime=2.5
        ovs=5
        if(Match_to_HCO):
            vel_prime=3.3
            ovs=3
    if(kl==5,km==5):
        Trunks=True
    else:
        Trunks=True




    try:
        Names=list(np.load(Name_File+".npy",allow_pickle=True))
        if(iterations!=np.shape(Names)[3]):
            del popasasd
    except Exception as e:

        print(e)
        print("initialized names")
        Names = list(np.empty((20,iterations,iterations,iterations,iterations,iterations,iterations),dtype=object))


    #Continuum image
    scCont = spectral_cube.Projection.from_hdu(fits.open(pathCont)[0])
    #Put this up here for the column density map
    metadata = {}
    metadata["distance"] = dist_cmz
    arc_per_pix_yc =  abs(scCont.hdu.header["CDELT1"]*3600.0 * u.arcsec)/u.pix
    arc_per_pix_xc =  abs(scCont.hdu.header["CDELT2"]*3600.0 * u.arcsec)/u.pix
    beam_majorc =  scCont.hdu.header["BMAJ"]*3600.0 * u.arcsec
    beam_minorc =  scCont.hdu.header["BMIN"]*3600.0 * u.arcsec
    beam_area_ratioc = beam_minorc*beam_majorc/arc_per_pix_yc/arc_per_pix_xc*1.13309#beam_area_ratioc = beam_minorc*beam_majorc/arc_per_pix_yc/arc_per_pix_xc#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    metadata['beam_area_ratioc']=beam_area_ratioc
    pc_per_pixelc = abs(scCont.hdu.header["CDELT1"])/180*np.pi*metadata['distance']/u.Mpc
    print(pc_per_pixelc,"MPC per pix")

    #Make subcube


    if(NM):
        Cube_Name_Load = 'Noise_Matched_'+"Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
    else:
        Cube_Name_Load = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
    path = Cube_Name_Load

    Qp = SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="radio") 
    Qp.allow_huge_operations=True

    Q = Qp.to(u.K)#Jy to Kelvin

    sc = Q.unmasked_copy()



    scW = sc.wcs[:][:][0]
    dat = sc.hdu.data[int(len(sc)*((1-Ram_Limiter)/2)):int(len(sc)*(Ram_Limiter + (1-Ram_Limiter)/2)),:,:]
    scn = sc[int(len(sc)*((1-Ram_Limiter)/2)):int(len(sc)*(Ram_Limiter + (1-Ram_Limiter)/2)),:,:]
    scF = scn
    datn = dat


    #m=.115

    #print(m,"Presumed Noise (K)")

    #Continuum image
    scCont = spectral_cube.Projection.from_hdu(fits.open(pathCont)[0])
    #Do the same thing to the continuum image
    scCont.allow_huge_operations=True
    scContW = scCont.reproject(scF.moment0().header)
    #m=.115


    scW = sc.wcs[:][:][0]




    #Do the same thing to the continuum image

    Continuum_Data  =scContW.hdu.data
    scWCS = scF.wcs[:][:][0]

    moment_0_sub  = scF.to(u.K).moment(order=0,how='slice')            # Calculate the Moment 0 map 



    ######Moment 0 for both
    ######and cont
    Make_Plot((LineN+" Moment 0"),"Moment 0 (K km/s)",moment_0_sub.hdu.data,0,np.nanmax(moment_0_sub.hdu.data),moment_0_sub.wcs,1,1,1,True)


    cSD = (Flux_to_Mass(scContW.hdu.data*u.Jy/u.beam/beam_area_ratioc*u.pix**2*u.beam)/(pc_per_pixelc*10**6*3.086*10**18*u.cm)**2)*(1.989*10**30*u.kg/u.M_sun)*Num_per_kg/u.kg #Flux goes to luminosity for a gaussian beam and goes to column density for square pixels, mass goes to kg to number





    rm=moment_0_sub.hdu.data/cSD
    rmU = rm*u.K*u.km/u.s#Just put back in the units
    rmU = np.array(rmU /(u.K*u.km/u.s)/u.cm**2*10**22,dtype='float64')#Now remove them all
    print(rmU)

    ######ratio

    bp = np.where(cSD<=.0001*10**22/u.cm**2)
    bp2 = np.where( moment_0_sub.hdu.data < .00022)
    Continuum_Data[bp] = np.nan
    cSD[bp]=np.nan
    rmU[bp]=np.nan
    rmU[bp2]=np.nan

    Non_nan=((datn[0,:,0:int(np.shape(datn)[2]/2)]>0)  | (datn[0,:,0:int(np.shape(datn)[2]/2)]<0 ))

    m = (np.nanstd(datn[0,:,0:int(np.shape(datn)[2]/2)],where= Non_nan)) #Noise K
    mp=m
    if(Abs_Level=="All"):
        pass
    if(Abs_Level=="None"):
        datn[np.where(datn<0)]=np.nan
    if(Abs_Level=="None (m)"):
        datn[np.where(datn<-m)]=np.nan
    if(Abs_Level=="None All Channels"):
        for lmi in range(len(datn)):
            bpP = np.where(datn[lmi]<-m)
            for lmj in range(len(datn)):
                datn[lmj][bpP]=np.nan
    if(Abs_Level=="No Clusters"):

        #IDs,RAs,Decs,HWHM  =    np.genfromtxt("NGC_Clusters_"+"IDs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"RAs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"Decs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"HWHM"+"")

        HWHM_rad,CDs_New,glons_New,glats_New,IDs_New   =    np.genfromtxt("CMZ_Clusters_"+"HWHM"+""),np.genfromtxt("CMZ_Clusters_"+"CD"+""),np.genfromtxt("CMZ_Clusters_"+"Glon"+""),np.genfromtxt("CMZ_Clusters_"+"Glat"+""),np.genfromtxt("CMZ_Clusters_"+"IDs"+"")

        datn=  Mask_Clusters_CMZ(HWHM_rad,scWCS,scF.hdu.header,datn,glons_New,glats_New,One_Pc=False,One_Pc_Size=1,HWHM_Fac=3)

    print(m,"Unmatched Noise (K)", "Abs = ",Abs_Level)


    Make_Plot("Column Density","(#/cm^2)",cSD,float(np.nanmin(cSD*u.cm**2)),float(np.nanmax(cSD*u.cm**2)),scCont.wcs,1,1,1,True)
    Make_Plot((LineN+" Moment 0/Column Density"),"Moment 0 over Column Density of the Continuum (K km/s/(#/cm^2))",rmU,np.nanmean(rmU)*.5,abs(np.nanmean(rmU))*8,scCont.wcs,1,1,1,True)

    print(np.nanmin(rmU))



    header = scF.hdu.header
    print()
    #make metadata for the dendrogram

    try:
        freq = header["RESTFREQ"]*u.Hz#
        metadata['wavelength']=299792458*u.m/header["RESTFREQ"]#
        print(1,freq,metadata['wavelength'])
    except:
        freq = header["RESTFRQ"]*u.Hz#
        metadata['wavelength']=299792458*u.m/header["RESTFRQ"]#
    metadata['data_unit'] =scF[0][0][0].unit# header['BUNIT']
    metadata['arc_per_pix_y'] =  abs(header["CDELT1"]*3600.0 * u.arcsec)/u.pix
    metadata['arc_per_pix_x'] =  abs(header["CDELT2"]*3600.0 * u.arcsec)/u.pix


    metadata['beam_major'] =  header["BMAJ"]*3600.0 * u.arcsec
    metadata['beam_minor'] =  header["BMIN"]*3600.0 * u.arcsec
    beam_area_ratio = metadata['beam_minor']*metadata['beam_major']/metadata['arc_per_pix_y']/metadata['arc_per_pix_x']*1.13309#beam_area_ratio = metadata['beam_minor']*metadata['beam_major']/metadata['arc_per_pix_y']/metadata['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    metadata['beam_area_ratio']=beam_area_ratio
    metadata['spatial_scale'] = np.sqrt(abs(header["CDELT1"])*u.degree**2*abs(header["CDELT2"]))
    #area_res = abs(header["CDELT1"])*abs(header["CDELT2"])*(np.pi/180*3.5)**2#mpc^2/pix^2
    #print(area_res,type(area_res))

    print(metadata['beam_minor'],metadata['beam_major'])
    print(beam_area_ratio)
    #metadata["wcs"] = wcs
    metadata["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#u.km/u.s

    metadata["vaxis"]=0
    metadata["wcsu"]=scF.wcs


    #for k3 in range(start_num):
    k3=2


    beam_req = Min_beam_req*(k3+1)
    if kl==5:
        beam_req=.9/3
    k4=0
    print(kl,km,k3,k4)
    pix_thresh_factor = k4+1
    k6=2
    delt_factor = (3/5+k6/5)

    k7=2
    noise_factor = (3/5+k7/5)
    if(kl==5):
        m=.115
    else:
        m=mp
    print(kl,km,k3,k4,k6,k7)
    print("actiev noise = ",m)
    try:



        Cube_Name_Save = Name+"_Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name_Save+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_'+str(beam_req)+"_beam_size_req_"+str(pix_thresh_factor)+'_times_beam_pixels_'+str(delt_factor)+"_delt_factor_"+str(noise_factor*5)+"_SNR_NEW.fits"



        Names[Num][kl][km][k3][k4][k6][k7] = Cube_Name_Save

        NameS = (Cube_Name_Save+"Sigmas")
        NameR = (Cube_Name_Save+"Radii")
        NameCol = (Cube_Name_Save+"_Column")
        NameLum = (Cube_Name_Save+"_Luminosities")

        NameFlux = (Cube_Name_Save+"Flux_Dense")
        NameRFF = (Cube_Name_Save+"Rad_For_Flux")

        print(Cube_Name_Save)

        

        np.save(Name_File,Names)

        d= astrodendro.Dendrogram.load_from(Cube_Name_Save+"_Dendrogram")
 

        vel,RA,Dec = scF.world[:,0,0]
        Rads,Sigmas,CD,Lumin,SIDS,MOM0FLUX,Distances,V_err = Dendro_Arrays(d,datn,vel,Continuum_Data,metadata,beam_size=Prime_Beam.value,beam_req=beam_req,Trunks=Trunks,max_size =max_size,edge_cases=Edge_cases )


        ##Analyze dendograms
        pc_per_pixel = abs(header["CDELT1"])/180*np.pi*metadata['distance']/u.Mpc

        sig_per_pixel=np.nan#metadata['spectral_resolution']/u.pix


        #Finds Size, Linewidth, Luminosity, and Column Density of a structure for each structure and returns them in [[][]] arrays [Leaves][Branches]


        from scipy.optimize import curve_fit
        from scipy.optimize import leastsq

        def func(R,a,b):
            return a*R**(b)
        try:
            popt, pcov = curve_fit(func, Rads[0]*10**6,Sigmas[0])
        except:
            popt, pcov = [np.nan,np.nan],[[np.nan,np.nan],[np.nan,np.nan]]
        try:
            poptB, pcovB = curve_fit(func,  Rads[1]*10**6,Sigmas[1])
        except:
            poptB, pcovB = [np.nan,np.nan],[[np.nan,np.nan],[np.nan,np.nan]]



        Rcon = np.concatenate((Rads[0],Rads[1]))


        Scon = np.concatenate((Sigmas[0],Sigmas[1]))
        Scon2 = np.concatenate((Sigmas[2],Sigmas[3]))
        print(np.shape(Rcon))

        poptCon, pcovCon = curve_fit(func, Rcon*10**6, Scon)










        xs=np.linspace(np.nanmin(Rcon*10**6),np.nanmax(Rcon*10**6),50)

        ysL,ysB,ysCon=func(xs,popt[0],popt[1]),func(xs,poptB[0],poptB[1]),func(xs,poptCon[0],poptCon[1])


        LuminCon = np.concatenate((Lumin[0],Lumin[1]))
        ColumnCon= np.concatenate((CD[0],CD[1]))

        MOM0FLUXcon = np.concatenate((MOM0FLUX[0],MOM0FLUX[1]))
        DistancesCon= np.concatenate((Distances[0],Distances[1]))
        print(Distances)

        #Radius Luminosity fit
        #Good pixels are places where the luminosity and therefore column density are above zero
        gp = np.where(LuminCon>0)
        print(np.shape(Rcon),np.shape(LuminCon))
        lgp = LuminCon[gp]
        radgp = Rcon[gp]*10**6

        gp2 = np.where(ColumnCon>0)
        cgp = ColumnCon[gp2]
        rgp=Scon[gp2]**2/Rcon[gp2]/10**6
        dgp=DistancesCon[gp2]
        radgpFORRAT = Rcon[gp2]*10**6
        siggpFORRAT = Scon[gp2]
        lumFORRAT = LuminCon[gp2]
        mom0FORRAT = MOM0FLUXcon[gp2]

        #print(np.shape(Rcon[gp]*10**6),np.shape(LuminCon[gp]))

        RLpopt,RLpcov = curve_fit(func, np.array(radgp,dtype='float64'),np.array(lgp,dtype='float64'),maxfev=999999,p0 = np.array([10**27,1],dtype='float64'),ftol = 10**-10)

        xsRL=np.linspace(np.nanmin(radgp),np.nanmax(radgp),50)
        ysRL = func(xsRL,RLpopt[0],RLpopt[1])
        #Column density to Size-linewidth
        CDpopt,CDpcov = curve_fit(func, np.array(cgp,dtype='float64') ,np.array(rgp,dtype='float64'),maxfev=9999099,p0 = np.array([10,.01],dtype='float64'),ftol = 10**-10)

        Dist_popt,Dist_pcov = curve_fit(func, np.array(DistancesCon,dtype='float64') ,np.array(Scon,dtype='float64'),maxfev=9999099,ftol = 10**-10)
        Dist_popt2,Dist_pcov2 = curve_fit(func, np.array(dgp,dtype='float64') ,np.array(rgp,dtype='float64'),maxfev=9999099,ftol = 10**-10)

        xsDist=np.linspace(np.nanmin(DistancesCon),np.nanmax(DistancesCon),50)
        xsDist2=np.linspace(np.nanmin(dgp),np.nanmax(dgp),50)
        ysDist=func(xsDist,Dist_popt[0],Dist_popt[1])
        ysDist2=func(xsDist2,Dist_popt2[0],Dist_popt2[1])

        gp3 = np.where(lumFORRAT>0)
        lumFORRAT = lumFORRAT[gp3]
        rgpFORRAT = rgp[gp3]
        mom0FORRAT = mom0FORRAT[gp3]

        xsCD=np.linspace(np.nanmin(cgp),np.nanmax(cgp),50)
        ysCD = func(xsCD,CDpopt[0],CDpopt[1])


        print(poptCon,pcovCon,"Size Linewidth All_structures:")
        print("a = ",poptCon[0],"+-",np.sqrt(pcovCon[0][0]))
        print("b = ",poptCon[1],"+-",np.sqrt(pcovCon[1][1]))


        print(RLpopt,RLpcov,"Radius to Luminosity:")
        print("c = ",RLpopt[0],"+-",np.sqrt(RLpcov[0][0]))
        print("d = ",RLpopt[1],"+-",np.sqrt(RLpcov[1][1]))



        print(CDpopt,CDpcov,"Column Density to linewidth^2/size:")
        print("e = ",CDpopt[0],"+-",np.sqrt(CDpcov[0][0]))
        print("f = ",CDpopt[1],"+-",np.sqrt(CDpcov[1][1]))



        gp4=np.where(MOM0FLUXcon>0)
        mom0FORFlux=MOM0FLUXcon[gp4]
        radgpFORFlux=Rcon[gp4]*10**6
        rgpFORFlux=Scon[gp4]**2/Rcon[gp4]/10**6


        '''

        '''
        fig = plt.figure(1,figsize=(40,40))


        axAlpha = pylab.subplot(5, 5, 7)
        axBeta = pylab.subplot(5, 5, 8)
        axGamma = pylab.subplot(5, 5, 9)
        axDelta = pylab.subplot(5, 5, 10)




        p1=d.plotter()
        p1.plot_tree(axAlpha)
        axAlpha.set_xlabel("Structure")
        axAlpha.set_ylabel("Flux (K)")
        axAlpha.set_title("Whole data set")



        #Plot contours for the top ten clusters


        
        scCropped =scF.moment0().hdu.data
        scCropped[np.where(scF.moment0().hdu.data<.22)]=np.nan
        print(np.nanmean(np.nanstd(scF.moment0().hdu.data))*1)
        scCropped[bp]=np.nan
        axDelta = pylab.subplot(5,5,8,projection=scF.moment0().wcs)
        #axBeta.imshow(scCropped.moment0().hdu.data, origin='lower', interpolation='nearest', cmap=plt.cm.Blues,norm=colors.LogNorm(vmin=5))
        imBeta = axDelta.imshow(scCropped, origin='lower', interpolation='nearest', cmap=plt.cm.Blues,vmin=0,vmax = np.nanmax(scCropped)*.7 )



        ColumnCon= np.concatenate((CD[0],CD[1]))
        SIDScon = np.concatenate((SIDS[0],SIDS[1]))
        nth =0# sorted(CD[0])[len(CD[0])-20]

        G1 = True

        RA = axDelta.coords[0]                                                                  # 
        Dec = axDelta.coords[1]

        RA.set_ticks(size=-3)                                                                                      
        Dec.set_ticks(size=-3) 
        RA.set_ticklabel(exclude_overlapping=True) 
        Dec.set_ticklabel(exclude_overlapping=True)                                                                                     
        pylab.xlabel('Right Ascension',fontsize=20,labelpad=1)                               
        pylab.ylabel('Declination',fontsize=20,labelpad=1)
        axDelta.tick_params(axis = 'both', which = 'major', labelsize = 15)    
        cb=pylab.colorbar(imBeta,fraction=0.016,pad=0.04)                                     
        cb.set_label(label="Moment 0 K km/s",fontsize=10,rotation=270,labelpad=20) 
        cb.ax.tick_params(which = 'major', labelsize = 10)   
        pylab.annotate(s=LineN,fontsize=10,xy=(.01,1.05),xycoords="axes fraction",c="black")  
        pylab.annotate(s="Highest Density",fontsize=10,xy=(.31,1.05),xycoords="axes fraction",c="red")  











        ax2 = pylab.subplot(5, 5, 1)
        ax3 = pylab.subplot(5, 5, 2)
        ax4 = pylab.subplot(5, 5, 3)

        xpcon = ax2.scatter(Rcon*10**6,Scon,label="All_structures",s=30,alpha=.7)


        p = ax2.plot(xs,ysCon)





        st="WDS: a="+ str(np.format_float_scientific(poptCon[0],1))+"+-"+str(np.format_float_scientific(np.sqrt(pcovCon[0][0]),1))+" b="+str(np.format_float_scientific(poptCon[1],1))+"+-"+str(np.format_float_scientific(np.sqrt(pcovCon[1][1]),2))
        ax2.annotate(s=st,fontsize=10,xy=(0.01,0.01),xycoords="axes fraction")

        ax2.annotate(s=r'$\sigma$ = $a*R^b$',fontsize=10,xy=(0.01,0.10),xycoords="axes fraction")
        ax2.set_yscale('log')
        ax2.set_xscale('log')
        ax2.set_ylim(np.nanmin(Scon),np.nanmax(Scon))
        ax2.set_xlabel("Size (pc)",fontsize=12)
        ax2.set_ylabel("Sigma (km/s)",fontsize=12)
        ax2.legend(prop={'size': 12},loc="upper right")
        ax2.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")

        #gp = np.where(FRs>0)







        xspL = ax4.scatter(radgpFORRAT,siggpFORRAT,label="All_structures",s=30)




        ax4.set_yscale('log')
        ax4.set_xscale('log')

        ax4.set_xlabel("Radius (pc)",fontsize=12)
        ax4.set_ylabel("sig (km/s)",fontsize=12)
        ax4.legend(prop={'size': 12},loc="upper right")






        xspF = ax3.scatter(radgpFORRAT,rgp,label="All_structures",s=30)

        ax3.annotate(s=st,fontsize=12,xy=(0.01,0.04),xycoords="axes fraction")
        ax3.set_yscale('log')
        ax3.set_xscale('log')
        ax3.set_ylim(np.nanmin(rgp),np.nanmax(rgp))
        ax3.set_xlabel("Rad (pc)",fontsize=12)
        ax3.set_ylabel("Linewidth Ratio (km^2/s^2/pc)",fontsize=12)
        ax3.legend(prop={'size': 12})





        ax5 = pylab.subplot(5, 5, 4)
        ax6 = pylab.subplot(5, 5, 5)
        ax7 = pylab.subplot(5, 5, 6)



        xpconhh = ax5.scatter(radgpFORRAT,cgp,label="All_structures",s=30,alpha=.7)
        ax5.set_yscale('log')
        ax5.set_xscale('log')
        ax5.set_xlabel("Size (pc)",fontsize=12)
        ax5.set_ylabel("Column Density (#/cm^2 > 7e22)",fontsize=12)
        ax5.legend(prop={'size': 12},loc="upper right")




        xspF2 = ax6.scatter(cgp,rgp,label="All_structures",s=30,alpha=.7)
        pF2 = ax6.plot(xsCD,ysCD)

        ax6.set_yscale('log')
        ax6.set_xscale('log')
        ax6.set_ylim(np.nanmin(rgp),np.nanmax(rgp))
        ax6.set_xlabel("Column Density (#/cm^2 > 7e22)",fontsize=12)
        ax6.set_ylabel("Linewidth Ratio (km^2/s^2/pc)",fontsize=12)
        ax6.legend(prop={'size': 12},loc="upper right")
        ax6.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")




        xspLu2 = ax7.scatter(radgp,lgp,label="All_structure Whole Data Set",s=30,alpha=.7)
        pFLLu2 = ax7.plot(xsRL,ysRL)

        st = "WDS, c = "+ str(np.format_float_scientific(RLpopt[0],3))+"+-"+str(np.format_float_scientific(np.sqrt(RLpcov[0][0]),3))+" d = "+str(np.format_float_scientific(RLpopt[1],3))+"+-"+str(np.format_float_scientific(np.sqrt(RLpcov[1][1]),3))
        ax7.annotate(s='L = $c*R^d$',fontsize=12,xy=(0.01,0.13),xycoords="axes fraction")
        ax7.annotate(s=st,fontsize=12,xy=(0.01,0.04),xycoords="axes fraction")

        ax7.set_yscale('log')
        ax7.set_xscale('log')
        ax7.set_ylim(np.nanmin(lgp),np.nanmax(lgp))
        ax7.set_xlabel("Radius (pc)",fontsize=9)
        ax7.set_ylabel("Luminosity (erg)",fontsize=9)
        ax7.legend(prop={'size': 12},loc="upper right")
        ax7.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")


        axLam = pylab.subplot(5, 5, 11)
        lum_rat = axLam.scatter(lumFORRAT/radgpFORRAT**2,rgpFORRAT,label="All_structure  ",s=30,alpha=.7)
        #Off by factors for area, simply using r^2

        axLam.set_yscale('log')
        axLam.set_xscale('log')
        axLam.set_xlabel("Lum/R^2 (erg/pc^2)",fontsize=9)
        axLam.set_ylabel("Linewidth Ratio (km/s)^2/pc",fontsize=9)
        axLam.legend(prop={'size': 12},loc="upper right")
        axLam.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")



        axDelta = pylab.subplot(5, 5, 10)






        mom0_rat = axDelta.scatter(mom0FORFlux/radgpFORFlux**2,rgpFORFlux,label="All_structure  ",s=30,alpha=.7)
        #Off by factors for area, simply using r^2

        axDelta.set_yscale('log')
        axDelta.set_xscale('log')
        axDelta.set_xlabel("Mom0 Flux/pc^2 (K km/s / R^2)",fontsize=9)
        axDelta.set_ylabel("Linewidth Ratio (km/s)^2/pc",fontsize=9)
        axDelta.legend(prop={'size': 12},loc="upper right")
        axDelta.annotate(s=LineN,fontsize=10,xy=(0.01,0.89),xycoords="axes fraction")



        axdis = pylab.subplot(5, 5, 12)
        axdis2 = pylab.subplot(5, 5, 13)
        axdis.scatter(DistancesCon,Scon)
        axdis2.scatter(dgp,rgp)
        axdis.plot(xsDist,ysDist)
        axdis2.plot(xsDist2,ysDist2)
        axdis2.set_yscale('log')
        axdis2.set_xscale('log')
        axdis.set_yscale('log')
        axdis.set_xscale('log')

        pylab.show()









        v1 = d.viewer()
        v1.show()


        #Must use text because np load is broken
        if(Abs_Level=="All"):
            Suffix=''
        if(Abs_Level=="None"):
            Suffix='_NA'
        if(Abs_Level=="No Clusters"):
            Suffix='_No_Clusters'
        if(Abs_Level=="None (m)"):
            Suffix='None_m'
        if(Abs_Level=="None All Channels"):
            Suffix='None_All_Channels'






        np.savetxt(os.path.join(savePath,NameCol+"_Leaves"+Suffix), CD[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameCol+"_Branches"+Suffix), CD[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameLum+"_Leaves"+Suffix), Lumin[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameLum+"_Branches"+Suffix), Lumin[1],fmt='%s')





        np.savetxt(os.path.join(savePath,NameS+"_Leaves"+Suffix), Sigmas[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameS+"_Branches"+Suffix), Sigmas[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameR+"_Leaves"+Suffix), Rads[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameR+"_Branches"+Suffix), Rads[1],fmt='%s')


        np.savetxt(os.path.join(savePath,NameFlux+"_Leaves"+Suffix), MOM0FLUX[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameFlux+"_Branches"+Suffix), MOM0FLUX[1],fmt='%s')



        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Leaves"+Suffix), Distances[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Branches"+Suffix), Distances[1],fmt='%s')






        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Leaves"+Suffix), V_err[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Branches"+Suffix), V_err[1],fmt='%s')

        SIDS=np.concatenate((SIDS[0],SIDS[1]))

        print(SIDS[len(SIDS)-3:len(SIDS)-1])

        
        
        import datetime
        now = str(datetime.datetime.now())
        string=now+ " "+ "Nico's method, common Gauss Smoothing (No Trunks,  min size=.9pc, delt = 1*m, m=.115K, SNR=5, max_size=21pc, G_width = sqrt(2.5^2-1),NM=true )"
        ax2.annotate(s=string,fontsize=18,xy=(0.01,1.0),xycoords="axes fraction")
    except Exception as e:
        print(e)
        print(kl,km,k3,k4)
        print("Failed")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)

        if(Abs_Level=="All"):
            Suffix=''
        if(Abs_Level=="None"):
            Suffix='_NA'
        if(Abs_Level=="No Clusters"):
            Suffix='_No_Clusters'
        if(Abs_Level=="None (m)"):
            Suffix='None_m'
        if(Abs_Level=="None All Channels"):
            Suffix='None_All_Channels'
        nans = [[np.nan,np.nan],[np.nan,np.nan]]



        np.savetxt(os.path.join(savePath,NameCol+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameCol+"_Branches"+Suffix), nans[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameLum+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameLum+"_Branches"+Suffix), nans[1],fmt='%s')





        np.savetxt(os.path.join(savePath,NameS+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameS+"_Branches"+Suffix), nans[1],fmt='%s')



        np.savetxt(os.path.join(savePath,NameR+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameR+"_Branches"+Suffix), nans[1],fmt='%s')


        np.savetxt(os.path.join(savePath,NameFlux+"_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,NameFlux+"_Branches"+Suffix), nans[1],fmt='%s')



        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"Distances_Branches"+Suffix), nans[1],fmt='%s')






        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Leaves"+Suffix), nans[0],fmt='%s')
        np.savetxt(os.path.join(savePath,Cube_Name_Save+"V_err_Branches"+Suffix), nans[1],fmt='%s')

